In [1]:
pip install split-folders

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
import urllib.request

In [16]:
import wget

In [17]:
!wget https://github.com/davidmallasen/LiveChess2FEN/releases/download/v0.1.0/ChessPiecesDataset.zip

!unzip /LinearAlgebraProject/ChessPiecesDataset.zip

zsh:1: command not found: wget
unzip:  cannot find or open /LinearAlgebraProject/ChessPiecesDataset.zip, /LinearAlgebraProject/ChessPiecesDataset.zip.zip or /LinearAlgebraProject/ChessPiecesDataset.zip.ZIP.


In [2]:
import splitfolders
splitfolders.ratio('/content/ChessPiecesDataset', output = "output", seed = 1, ratio = (.8, .1, .1))

ValueError: The provided input folder "/content/ChessPiecesDataset" does not exists.

In [ ]:
import os
base_dir = '/content/output/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
from tensorflow.keras.optimizers import SGD
batch_size = 256
opt = SGD(learning_rate=0.01, momentum=0.9)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# randomly rotate the images to allow the model to learn variations in orientation
datagen = ImageDataGenerator(
        rotation_range=5,
        # width_shift_range=0.1,
        # height_shift_range=0.1,
        rescale=1./255,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Model
from keras.applications.imagenet_utils import decode_predictions
from os import path
# Define the stopping callbacks
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 20)

In [ ]:
shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet201

shape = (224, 224)
train_gen = datagen.flow_from_directory(train_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=True)
valid_gen = test_datagen.flow_from_directory(validation_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)
test_gen = test_datagen.flow_from_directory(test_dir, target_size = shape, batch_size = batch_size, class_mode = 'categorical', color_mode = 'rgb', shuffle=False)

base_model = DenseNet201(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False    

# Establish new fully connected block
x = base_model.output
x = Flatten()(x)  # flatten from convolution tensor output  
x = Dense(1000, activation='relu')(x) # number of layers and units are hyperparameters, as usual
predictions = Dense(13, activation='softmax')(x) # should match # of classes predicted

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
epochs = 5
if os.path.exists('model_DenseNet201.h5'):
    model = tf.keras.models.load_model('model_DenseNet201.h5')
    print("Model loaded")
    history = model.fit(
    train_gen, 
    epochs=epochs,
    verbose = 1,
    validation_data=test_gen,
    callbacks = [es]
    )
    model.save('model_DenseNet201.h5') 
else:
    history = model.fit(
        train_gen, 
        epochs=epochs,
        verbose = 1,
        validation_data=test_gen,
        callbacks = [es]
        )
    model.save('model_DenseNet201.h5')

In [ ]:
model = tf.keras.models.load_model('model_DenseNet201.h5')
_, acc = model.evaluate(valid_gen, verbose=0)
print('> %.3f' % (acc * 100.0))